In [ ]:
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
# !tar -xzvf ta-lib-0.4.0-src.tar.gz
# %cd ta-lib
# !./configure --prefix=/usr
# !make
# !make install
# !pip install Ta-Lib

In [ ]:
import sys
!{sys.executable} -m pip install phik

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 KB 10.6 MB/s eta 0:00:00


In [ ]:
# pip install yfinance

In [ ]:
import numpy as np
import pandas as pd
# import talib as ta
from prophet import Prophet
import yfinance as yf
import matplotlib.pyplot as plt

import statsmodels
from statsmodels.tsa.stattools import coint
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.vector_ar.var_model import VAR
from sklearn.preprocessing import MinMaxScaler
#from pykalman import KalmanFilter
np.random.seed(107)
#import pywt

import sys
from datetime import datetime

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import plotly.offline as py
import plotly.graph_objs as go
import seaborn as sns
#py.init_notebook_mode(connected=True)
%matplotlib inline

# Load data and compute metrics

In [ ]:
def load_data():
    
    #read data
    raw_data = pd.read_csv("/content/drive/MyDrive/IAQF/fundamentals.csv", low_memory = False);
    
    #get the keys (indices)
    indices = raw_data.keys()
    
    #rename the columns to features
    raw_data.rename(columns = raw_data.iloc[0], inplace = True)
    
    #drop columns from data values
    raw_data.drop([0], inplace = True)
    
    #convert dates column to date format
    raw_data["Dates"] = pd.to_datetime(raw_data['Dates'], format = "%d-%m-%Y")
    #set dates as index
    raw_data.set_index("Dates", inplace = True)
    
    #extract list of indices
    indices = indices.to_list()
    indices = indices[1:]
    
    final_indices = []
    
    for i in range(len(indices)):
        if i%23 == 0:
            final_indices.append(indices[i])
     
    columns = []
    for i in raw_data.keys():
        columns.append(i)
    
    #make the columns unique
    temp_columns = columns.copy()
    for i in range(len(columns)):
        temp_columns[i] = temp_columns[i] + " "+ final_indices[int(i/23)]
    
    #drop first row
    raw_data.drop("01-01-2010", inplace = True)
    
    #rename columns
    raw_data.columns = temp_columns
    
    #change data type to float
    raw_data = raw_data.astype(float)
                                       
    #get list of features
    final_columns = [];
    [final_columns.append(x) for x in columns if x not in final_columns];
    
    #make a 3d dataframe
    final_dict = {}
    for i in range(len(raw_data.keys())):
        final_dict[final_indices[int(i/23)], final_columns[i%23]] = raw_data[raw_data.keys()[i]]
        
    final_raw_data = pd.DataFrame(final_dict)
    
    #get all the indices where all data is present
    temp_dict = {}
    for outer_key in final_indices:
        if len(final_raw_data[outer_key].dropna(axis = 1).keys()) == 23:
            for inner_key in final_raw_data[outer_key].keys():
                temp_dict[outer_key, inner_key] = final_raw_data[outer_key][inner_key]
    
    final_df = pd.DataFrame(temp_dict)
    
    return final_df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = load_data()

In [ ]:
clusters = pd.read_csv("/content/cluster_Index_data_latest.csv")

In [ ]:
clusters.drop("Unnamed: 0", axis = 1, inplace = True)

In [ ]:
clusters

,Index,Cluster
0,SPX Index,9
1,INDU Index,6
2,SXXP Index,9
3,DAX Index,3
4,NDX Index,3
...,...,...
91,TPREAL Index,2
92,OMXS30B Index,2
93,SPR Index,4
94,SBX Index,4


In [ ]:
clusters.groupby(['Cluster']).count()

,Index
Cluster,
0,12
1,20
2,8
3,10
4,11
5,6
6,10
7,8
8,5


In [ ]:
cluster_list = []

for i in range(10):
    cluster_list.append([])

for i in range(len(clusters)):
    cluster_list[clusters.iloc[i][1]].append(clusters.iloc[i][0])

In [ ]:
cluster_list

[['SXXR Index',
  'SGX Index',
  'SVX Index',
  'M1WD Index',
  'SPTR500N Index',
  'SXTP Index',
  'SMIM Index',
  'MXWO000G Index',
  'SET50 Index',
  'MXEM Index',
  'NSEIT Index',
  'MXBR Index'],
 ['SET Index',
  'SXPP Index',
  'DJI Index',
  'RAY Index',
  'S5INFT Index',
  'RLG Index',
  'MXUS Index',
  'MDAX Index',
  'IPSA Index',
  'MXAP Index',
  'JALSH Index',
  'SXNP Index',
  'S5HLTH Index',
  'SXRP Index',
  'OSEBX Index',
  'SX6P Index',
  'BE500 Index',
  'SXKP Index',
  'IBOVE Index',
  'S5TELS Index'],
 ['NMX Index',
  'SXOP Index',
  'SAX Index',
  'PSI20 Index',
  'SPIEX Index',
  'IMOEX Index',
  'TPREAL Index',
  'OMXS30B Index'],
 ['DAX Index',
  'NDX Index',
  'IBOV Index',
  'CAC Index',
  'MXEF Index',
  'MXWD Index',
  'SPXL1 Index',
  'OMX Index',
  'MEXBOL Index',
  'SPI Index'],
 ['NYA Index',
  'NDQ Index',
  'SX3P Index',
  'OEX Index',
  'MXLA Index',
  'SBF120 Index',
  'SCXP Index',
  'MXWOU Index',
  'MXWO0IT Index',
  'SPR Index',
  'SBX Index'],


In [ ]:
def find_cointegrated_pairs(data, indice_list):
    
    n = len(indice_list)
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    
    pairs = {}
    
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[indice_list[i]]["PX_LAST"]
            S2 = data[indice_list[j]]["PX_LAST"]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.01:
                pairs[(indice_list[i], indice_list[j])] = [score, pvalue]
    return score_matrix, pvalue_matrix, pairs

In [ ]:
from sklearn.linear_model import LinearRegression

def find_maxr2_pairs(data, indice_list):

    n = len(indice_list)
    score_matrix = np.zeros((n, n))
    
    pairs = {}
    
    for i in range(n):
        for j in range(i+1, n):
            # convert to uniform/quantile distribution, perform LR, find R^2
            model = LinearRegression()
            X = data[indice_list[i]][["PX_LAST"]]
            y = data[indice_list[j]]["PX_LAST"]
            Ux = X.rank(pct=True)
            Uy = y.rank(pct=True)
            model = model.fit(Ux, Uy)
            r2_score = model.score(Ux, Uy)
            score_matrix[i, j] = r2_score
            if r2_score >= 0.95:
                pairs[(indice_list[i], indice_list[j])] = r2_score
    return score_matrix, pairs

In [ ]:
from sklearn.feature_selection import mutual_info_regression

def find_mi_pairs(data, indice_list):

    n = len(indice_list)
    score_matrix = np.zeros((n, n))
    
    pairs = {}
    
    for i in range(n):
        for j in range(i+1, n):
            # convert to uniform/quantile distribution, perform LR, find R^2
            X = data[indice_list[i]][["PX_LAST"]]
            y = data[indice_list[j]]["PX_LAST"]
            mi = mutual_info_regression(X, y)
            score_matrix[i, j] = mi
            if mi >= 2:
                pairs[(indice_list[i], indice_list[j])] = mi
    return score_matrix, pairs

In [ ]:
def find_phik_pairs(data, indice_list):
    
    pairs = {}
    px_last_data = small_data[indice_list].xs('PX_LAST', axis=1, level=1, drop_level=True)
    interval_cols = px_last_data.columns
    phik_overview = px_last_data.phik_matrix(interval_cols=interval_cols)
    phik_overview.round(4)
    upper_t = phik_overview.where(np.triu(np.ones(phik_overview.shape)).astype(np.bool))
    stacked_series = upper_t.stack()
    filtered = stacked_series[(stacked_series>0.95) & (stacked_series != 1)]
    pairs = list(zip(filtered.index, filtered.values))
    return phik_overview, pairs

In [ ]:
small_data = data["2021-01-01":]

In [ ]:
final_scores = []
final_pvalues = []
final_pairs = []

for indice_list in cluster_list:
    scores, pvalues, pairs = find_cointegrated_pairs(small_data,indice_list)
    final_scores.append(scores)
    final_pvalues.append(pvalues)
    final_pairs.append(pairs)
    # plt.figure(figsize = (8,8))
    # sns.heatmap(pvalues,  cmap='RdYlGn_r', mask = (pvalues >= 0.98), xticklabels = indice_list, yticklabels = indice_list)
    # pyplot.show()

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/stattools.py:1605: CollinearityWarning:

y0 and y1 are (almost) perfectly colinear.Cointegration test is not reliable in this case.



In [ ]:
final_pairs

[{},
 {('DJI Index', 'BE500 Index'): [-4.183370967599206, 0.003865208258288035],
  ('RAY Index', 'IPSA Index'): [-4.037998623566766, 0.006334289159482081],
  ('MDAX Index', 'IPSA Index'): [-4.105683944663578, 0.0050465124352194355],
  ('IPSA Index', 'SXNP Index'): [-3.951654236434797, 0.00840617300566543],
  ('IPSA Index', 'SXRP Index'): [-4.02501254981698, 0.006613008261378933],
  ('IPSA Index', 'S5TELS Index'): [-5.100994480294302,
   0.00010690473058597416]},
 {},
 {('DAX Index', 'OMX Index'): [-4.563306599851863, 0.0009619562474744305]},
 {},
 {},
 {('SPXT Index', 'SPTR Index'): [-inf, 0.0]},
 {},
 {},
 {}]

In [ ]:
r2_final_scores = []
r2_final_pairs = []
print('R-squarred')

for indice_list in cluster_list:
    scores, pairs = find_maxr2_pairs(small_data, indice_list)
    r2_final_scores.append(scores)
    r2_final_pairs.append(pairs)
    # plt.figure(figsize = (8,8))
    # sns.heatmap(scores,  cmap='RdYlGn_r', mask = (scores <= 0.50), xticklabels = indice_list, yticklabels = indice_list)
    # pyplot.show()

R-squarred


In [ ]:
r2_final_pairs

[{('M1WD Index', 'MXWO000G Index'): 0.9585672442145772},
 {('RAY Index', 'RLG Index'): 0.9578265034504089,
  ('RAY Index', 'MXUS Index'): 0.9929343405073495,
  ('S5INFT Index', 'MXUS Index'): 0.9551134522310114},
 {},
 {},
 {('NDQ Index', 'MXWO0IT Index'): 0.9889568741434022,
  ('OEX Index', 'SPR Index'): 0.983290754263528},
 {},
 {('SPXT Index', 'SPTR Index'): 0.9999999635404472},
 {('RIY Index', 'SPXL2 Index'): 0.9729163857071447},
 {},
 {('SXXP Index', 'MXEU Index'): 0.9881989659881345}]

In [ ]:
mi_final_scores = []
mi_final_pairs = []
print('Mutual Info')

for indice_list in cluster_list:
    scores, pairs = find_mi_pairs(small_data, indice_list)
    mi_final_scores.append(scores)
    mi_final_pairs.append(pairs)
    # plt.figure(figsize = (8,8))
    # sns.heatmap(scores,  cmap='RdYlGn_r', mask = (scores <= 0.50), xticklabels = indice_list, yticklabels = indice_list)
    # pyplot.show()

Mutual Info


In [ ]:
mi_final_pairs

[{('SGX Index', 'MXWO000G Index'): array([2.05157349])},
 {('RAY Index', 'MXUS Index'): array([3.06917425]),
  ('S5INFT Index', 'RLG Index'): array([2.0878758]),
  ('RLG Index', 'MXUS Index'): array([2.02221251])},
 {('SAX Index', 'OMXS30B Index'): array([2.03612375])},
 {},
 {('NDQ Index', 'MXWO0IT Index'): array([2.52560906]),
  ('OEX Index', 'SPR Index'): array([2.46679258])},
 {},
 {('SPXT Index', 'SPTR Index'): array([5.03920844])},
 {('RIY Index', 'SPXL2 Index'): array([2.97280539])},
 {},
 {('SXXP Index', 'MXEU Index'): array([2.99345167])}]

In [ ]:
print('Phik')
from phik.report import plot_correlation_matrix
phik_final_scores = []
phik_final_pairs = []

for indice_list in cluster_list:
    phik_overview, pairs = find_phik_pairs(small_data, indice_list)
    phik_final_scores.append(phik_overview)
    phik_final_pairs.append(pairs)

    # plot_correlation_matrix(phik_overview.values, 
    #                         x_labels=phik_overview.columns, 
    #                         y_labels=phik_overview.index, 
    #                         vmin=0, vmax=1, color_map="Greens", 
    #                         title=r"correlation $\phi_K$", 
    #                         fontsize_factor=1.5, 
    #                         figsize=(10, 8))
    # plt.tight_layout()

Phik


<ipython-input-97-9a533d8b6a92>:8: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [ ]:
phik_final_pairs

[[(('M1WD Index', 'MXWO000G Index'), 0.9544048820484774)],
 [(('RAY Index', 'RLG Index'), 0.9629556519193628),
  (('RAY Index', 'MXUS Index'), 0.9931691375050243),
  (('S5INFT Index', 'MXUS Index'), 0.9549469885748979),
  (('RLG Index', 'MXUS Index'), 0.9611009416197959)],
 [(('SXOP Index', 'SAX Index'), 0.9545310747581168)],
 [(('DAX Index', 'OMX Index'), 0.9571301172702076)],
 [(('NDQ Index', 'MXWO0IT Index'), 0.9775397354594296),
  (('OEX Index', 'SPR Index'), 0.9821969589582018),
  (('SCXP Index', 'MXWOU Index'), 0.9631242444796778)],
 [],
 [],
 [(('RIY Index', 'SPXL2 Index'), 0.9807432938287237)],
 [],
 [(('SXXP Index', 'MXEU Index'), 0.9889344097122742)]]

In [ ]:
import statsmodels.api as sm

In [ ]:
S1 = small_data['SET50 Index']['PX_LAST']
S2 = small_data['NSEIT Index']['PX_LAST']

S1 = sm.add_constant(S1)
results = sm.OLS(S2, S1).fit()

In [ ]:
b = results.params["PX_LAST"]
spread = S2 - b*S1["PX_LAST"]

In [ ]:
plt.plot(spread)